In [1]:
import pandas as pd

df = pd.read_csv("data/Data.csv")
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


### 大模型语言链

In [4]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import utils_zh

_ = load_dotenv(find_dotenv())

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
    )

# langchain是一种快速开发应用程序框架，组件可以链式组合

In [5]:
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains.llm import LLMChain

In [6]:
llm = ChatOpenAI(temperature=0.0)

In [7]:
prompt = ChatPromptTemplate.from_template(
    "描述制造{product}的一个公司的最佳名称是什么？"
)
chain = LLMChain(llm=llm, prompt=prompt)
product="大号床单套装"
chain.run(product)

'“豪华床品集团”'

### 顺序链

In [8]:
from langchain.chains.sequential import SimpleSequentialChain
llm = ChatOpenAI(temperature=0.9)

In [9]:
first_prompt = ChatPromptTemplate.from_template(
    "描述制造{product}的一个公司的最好的名称是什么"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)
second_prompt = ChatPromptTemplate.from_template(
    "写一个20字的描述对于下面这个\
        公司：{company_name}的"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)
product = "大号床单套装"
overall_simple_chain.run(product)




> Entering new SimpleSequentialChain chain...
豪华纺织品有限公司
豪华纺织品有限公司致力于生产和销售高品质、豪华的纺织产品，以打造舒适、奢华的生活方式。

> Finished chain.


'豪华纺织品有限公司致力于生产和销售高品质、豪华的纺织产品，以打造舒适、奢华的生活方式。'

In [10]:
from langchain.chains.sequential import SequentialChain
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains.llm import LLMChain

llm = ChatOpenAI(temperature=0.9)

简单顺序链（单输入单输出）
顺序链（多输入或多输出）

In [12]:
first_prompt = ChatPromptTemplate.from_template(
    "把下面的评论review翻译成英文："
    "\n\n{Review}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

second_prompt = ChatPromptTemplate.from_template(
    "请你用一句话来总结下面的评论Review："
    "\n\n{English_Review}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

third_prompt = ChatPromptTemplate.from_template(
    "下面的评论review使用什么语言：\n\n{Review}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

fourth_prompt = ChatPromptTemplate.from_template(
    "使用特定的语言对下面的总结写一个后续回复："
    "\n\n总结：{summary}\n\n语言：{language}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message")
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","language","followup_message"],
    verbose=True
)

review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer finds the taste of the product mediocre and suspects it may be an old batch or counterfeit.',
 'language': 'Ce commentaire est en français.',
 'followup_message': "Je suis désolé d'entendre que vous avez trouvé le goût du produit moyen. Nous sommes très attentifs à la qualité de nos produits et nous vous assurons que nous n'utilisons que des ingrédients de la plus haute qualité. Il est possible que vous ayez reçu un lot ancien ou contrefait, et pour cela nous vous invitons à nous contacter directement pour résoudre ce problème. Votre satisfaction est notre priorité et nous ferons tout notre possible pour rectifier l

#### 路由链

In [22]:
from langchain.chains.router import MultiPromptChain  #导入多提示链
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model='gpt-4o', temperature=0)

In [14]:
physics_template = """你是一个非常聪明的物理专家。 \
你擅长用一种简洁并且易于理解的方式去回答问题。\
当你不知道问题的答案时，你承认\
你不知道.

这是一个问题:
{input}"""

#第二个提示适合回答数学问题
math_template = """你是一个非常优秀的数学家。 \
你擅长回答数学问题。 \
你之所以如此优秀， \
是因为你能够将棘手的问题分解为组成部分，\
回答组成部分，然后将它们组合在一起，回答更广泛的问题。

这是一个问题：
{input}"""

#第三个适合回答历史问题
history_template = """你是以为非常优秀的历史学家。 \
你对一系列历史时期的人物、事件和背景有着极好的学识和理解\
你有能力思考、反思、辩证、讨论和评估过去。\
你尊重历史证据，并有能力利用它来支持你的解释和判断。

这是一个问题:
{input}"""

#第四个适合回答计算机问题
computerscience_template = """ 你是一个成功的计算机科学专家。\
你有创造力、协作精神、\
前瞻性思维、自信、解决问题的能力、\
对理论和算法的理解以及出色的沟通技巧。\
你非常擅长回答编程问题。\
你之所以如此优秀，是因为你知道  \
如何通过以机器可以轻松解释的命令式步骤描述解决方案来解决问题，\
并且你知道如何选择在时间复杂性和空间复杂性之间取得良好平衡的解决方案。

这还是一个输入：
{input}"""


In [15]:
prompt_infos = [
    {
        "名字": "物理学", 
        "描述": "擅长回答关于物理学的问题", 
        "提示模板": physics_template
    },
    {
        "名字": "数学", 
        "描述": "擅长回答数学问题", 
        "提示模板": math_template
    },
    {
        "名字": "历史", 
        "描述": "擅长回答历史问题", 
        "提示模板": history_template
    },
    {
        "名字": "计算机科学", 
        "描述": "擅长回答计算机科学问题", 
        "提示模板": computerscience_template
    }
]

In [16]:
#目标链：不同问题类型及对应的目标链模板
destination_chains={}
for p_info in prompt_infos:
    name = p_info["名字"]
    prompt_template = p_info["提示模板"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['名字']}:{p['描述']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [21]:
destinations_str

'物理学:擅长回答关于物理学的问题\n数学:擅长回答数学问题\n历史:擅长回答历史问题\n计算机科学:擅长回答计算机科学问题'

In [19]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [20]:
# 对完成任务进行分类（路由选择链路），输出的格式

# 多提示路由模板
MULTI_PROMPT_ROUTER_TEMPLATE = """给语言模型一个原始文本输入，\
让其选择最适合输入的模型提示。\
系统将为您提供可用提示的名称以及最适合改提示的描述。\
如果你认为修改原始输入最终会导致语言模型做出更好的响应，\
你也可以修改原始输入。


<< 格式 >>
返回一个带有JSON对象的markdown代码片段，该JSON对象的格式如下：
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}
```


记住：“destination”必须是下面指定的候选提示名称之一，\
或者如果输入不太适合任何候选提示，\
则可以是 “DEFAULT” 。
记住：如果您认为不需要任何修改，\
则 “next_inputs” 可以只是原始输入。

<< 候选提示 >>
{destinations}

<< 输入 >>
{{input}}

<< 输出 (记得要包含 ```json)>>

样例:
<< 输入 >>
"什么是黑体辐射?"
<< 输出 >>
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}
```

"""

In [23]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [24]:
#多提示链
chain = MultiPromptChain(router_chain=router_chain,#路由链
                         destination_chains=destination_chains,#目标链路
                         default_chain=default_chain,
                         verbose=True)


In [25]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


/Applications/anaconda3/envs/llm-universe/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


物理学: {'input': 'What is black body radiation?'}
> Finished chain.


'黑体辐射是指一个理想化的物体（称为黑体）在热平衡状态下发出的电磁辐射。黑体是一个完全吸收所有入射辐射的物体，不反射或透射任何光。根据热力学原理，黑体在吸收能量的同时也会以特定的频谱和强度发射辐射，这种辐射仅依赖于黑体的温度。\n\n黑体辐射的经典描述是由普朗克定律给出的，它表明辐射的能量分布与温度有关。随着温度的升高，黑体辐射的峰值波长会向更短的波长移动，这就是维恩位移定律的内容。此外，斯特藩-玻尔兹曼定律描述了黑体辐射的总能量与温度的四次方成正比。\n\n黑体辐射的研究在历史上非常重要，因为它促成了量子力学的诞生。'

In [26]:
chain.run("你知道李白是谁嘛?")



> Entering new MultiPromptChain chain...


/Applications/anaconda3/envs/llm-universe/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


历史: {'input': '李白是谁？'}
> Finished chain.


'李白是中国唐代著名的诗人，被誉为“诗仙”。他生于701年，卒于762年，字太白，号青莲居士。李白以其豪放不羁的个性和卓越的诗才而闻名，他的诗歌以想象丰富、语言奔放、情感真挚著称。李白的作品多描绘自然风光、抒发个人情怀，常常表现出一种超脱世俗的浪漫主义精神。\n\n李白一生仕途坎坷，曾短暂供职于唐玄宗的翰林院，但因不满官场生活而辞职，之后过着漫游四方的生活。他与杜甫并称为“李杜”，是中国文学史上最重要的诗人之一。李白的诗作对后世影响深远，其代表作包括《将进酒》、《行路难》、《早发白帝城》等。'

In [27]:
chain.run("为什么我们身体里的每个细胞都包含DNA？")




> Entering new MultiPromptChain chain...


/Applications/anaconda3/envs/llm-universe/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


None: {'input': '为什么我们身体里的每个细胞都包含DNA？'}
> Finished chain.


'我们身体里的每个细胞都包含DNA，因为DNA是遗传信息的载体，负责指导细胞的生长、发育、功能和繁殖。以下是一些原因：\n\n1. **遗传信息的传递**：DNA包含了所有遗传信息，这些信息决定了生物体的特征和功能。每个细胞都需要这些信息来执行其特定的功能。\n\n2. **细胞分裂和生长**：在细胞分裂过程中，DNA需要被复制，以确保每个新细胞都具有完整的遗传信息。这对于生长、修复和繁殖是至关重要的。\n\n3. **蛋白质合成**：DNA中的基因编码蛋白质，而蛋白质是细胞功能的执行者。每个细胞需要能够合成特定的蛋白质来维持其功能和结构。\n\n4. **细胞特化**：虽然每个细胞都包含相同的DNA，但不同类型的细胞会根据需要激活不同的基因。这种基因表达的调控使得细胞能够特化为不同的类型，如肌肉细胞、神经细胞等。\n\n5. **修复和维护**：DNA还包含修复机制的信息，以帮助细胞修复损伤并维持基因组的完整性。\n\n总之，DNA在每个细胞中是必不可少的，因为它提供了细胞所需的所有信息，以维持生命的基本过程。'